In [1]:
#####設定讀取資料的年份與時段#####

year = 104    #110年或104年
rush_hour = '昏峰'    #'晨峰'或'昏峰'

inputLocation = f"{year}_現況_{rush_hour}/"

################################

import pandas as pd
import numpy as np

def calculate_mape(actual, predicted):
    """計算 MAPE (Mean Absolute Percentage Error)"""
    if actual == 0:
        return 0.0  # 避免除以零
    return abs((actual - predicted) / actual) * 100

def calculate_overall_mape(actual_values, predicted_values):
    """計算整體 MAPE"""
    valid_pairs = [(a, p) for a, p in zip(actual_values, predicted_values) if a != 0]
    if not valid_pairs:
        return 0.0
    
    mape_values = [calculate_mape(a, p) for a, p in valid_pairs]
    return np.mean(mape_values)

def sort_group_names(group_names):
    """按照 SL1, SL2, ... 順序排序 group 名稱"""
    def extract_number(group_name):
        try:
            # 提取 SL 後面的數字
            if isinstance(group_name, str) and group_name.startswith('SL'):
                return int(group_name[2:])  # 取 SL 後面的數字部分
            else:
                return float('inf')  # 非 SL 格式的放在最後
        except:
            return float('inf')
    
    return sorted(group_names, key=extract_number)

# ...existing code...
# 讀取 Excel 檔案，處理空值
df = pd.read_excel("TRTS-4S屏柵線.xlsx", sheet_name=None)
sheet1 = df["節點編號"]

# 處理 sheet1 中的空值
sheet1 = sheet1.fillna(0)  # 將空值填充為 0
sheet1 = sheet1.replace('', 0)  # 將空字串替換為 0

if rush_hour == '晨峰':
    real_flow = df["晨峰實際流量"]
elif rush_hour == '昏峰':
    real_flow = df["昏峰實際流量"]

# 處理 real_flow 中的空值
real_flow = real_flow.fillna(0)
real_flow = real_flow.replace('', 0)

real_flow = real_flow.set_index("group")

ue_results = pd.read_csv(inputLocation + f"{year}_現況_{rush_hour}_UE_results.dat", sep="\t")
ue_results.rename(columns={"tailNode": "A", "headNode": "B"}, inplace=True)

ue_results["link_name"] = ue_results.apply(lambda row: f"{int(row['A'])}_{int(row['B'])}", axis=1)

# 建立 link_name 時處理可能的空值
def safe_link_name(a, b):
    """安全地建立 link_name，處理空值"""
    try:
        if pd.isna(a) or pd.isna(b) or a == 0 or b == 0:
            return "0_0"  # 空值或 0 的情況
        return f"{int(a)}_{int(b)}"
    except (ValueError, TypeError):
        return "0_0"

sheet1["link_name1"] = sheet1.apply(lambda row: safe_link_name(row['A1'], row['B1']), axis=1)
sheet1["link_name2"] = sheet1.apply(lambda row: safe_link_name(row['A2'], row['B2']), axis=1)

# 合併資料時處理空值
sheet1 = sheet1.merge(ue_results[["link_name", "UE_flow", "capacity"]], 
                      left_on="link_name1", right_on="link_name", how="left")
sheet1 = sheet1.merge(ue_results[["link_name", "UE_flow", "capacity"]], 
                      left_on="link_name2", right_on="link_name", how="left", suffixes=('_1', '_2'))

# 處理合併後的空值
sheet1['UE_flow_1'] = sheet1['UE_flow_1'].fillna(0)
sheet1['UE_flow_2'] = sheet1['UE_flow_2'].fillna(0)

# 保留需要的欄位
sheet1 = sheet1[["group","link", "UE_flow_1", "UE_flow_2"]]

# 處理 group 可能為空值的情況
sheet1 = sheet1.dropna(subset=['group'])  # 移除 group 為空值的行

group_names = sheet1['group'].unique()
groups = {i: {'UE_flow_1': 0, 'UE_flow_2': 0} for i in group_names}

for index, row in sheet1.iterrows():
    group = row['group']
    # 確保流量值不是 NaN
    ue_flow_1 = row['UE_flow_1'] if not pd.isna(row['UE_flow_1']) else 0
    ue_flow_2 = row['UE_flow_2'] if not pd.isna(row['UE_flow_2']) else 0
    
    groups[group]['UE_flow_1'] += ue_flow_1
    groups[group]['UE_flow_2'] += ue_flow_2

# UE_flow_1 代表(往南/東)流量， UE_flow_2 代表(往北/西)流量
for group, flows in groups.items():
    try:
        # 檢查 real_flow 中是否有該 group 的資料
        if group not in real_flow.index:
            print(f"【{group}】 資料不完整，跳過計算")
            continue
            
        # 安全地取得實際流量數據
        name = real_flow.loc[group, 'name'] if 'name' in real_flow.columns else f"Group {group}"
        south_east = real_flow.loc[group, '南/東'] if '南/東' in real_flow.columns and not pd.isna(real_flow.loc[group, '南/東']) else 1
        north_west = real_flow.loc[group, '北/西'] if '北/西' in real_flow.columns and not pd.isna(real_flow.loc[group, '北/西']) else 1
        
        # 計算差異百分比，避免除以零
        model_diff_se = (flows['UE_flow_1'] - south_east) / south_east * 100 if south_east != 0 else 0
        model_diff_nw = (flows['UE_flow_2'] - north_west) / north_west * 100 if north_west != 0 else 0
        
        # CUBE 差異計算（如果有 CUBE 資料）
        cube_se = real_flow.loc[group, 'CUBE南/東'] if 'CUBE南/東' in real_flow.columns and not pd.isna(real_flow.loc[group, 'CUBE南/東']) else south_east
        cube_nw = real_flow.loc[group, 'CUBE北/西'] if 'CUBE北/西' in real_flow.columns and not pd.isna(real_flow.loc[group, 'CUBE北/西']) else north_west
        
        cube_diff_se = (cube_se - south_east) / south_east * 100 if south_east != 0 else 0
        cube_diff_nw = (cube_nw - north_west) / north_west * 100 if north_west != 0 else 0
        
    except Exception as e:
        print(f"【{group}】 處理時發生錯誤: {e}")
        continue

# 處理 group 可能為空值的情況
sheet1 = sheet1.dropna(subset=['group'])  # 移除 group 為空值的行

group_names = sheet1['group'].unique()
# 按照 SL1, SL2, ... 順序排序
sorted_group_names = sort_group_names(group_names)

groups = {i: {'UE_flow_1': 0, 'UE_flow_2': 0} for i in sorted_group_names}

for index, row in sheet1.iterrows():
    group = row['group']
    # 確保流量值不是 NaN
    ue_flow_1 = row['UE_flow_1'] if not pd.isna(row['UE_flow_1']) else 0
    ue_flow_2 = row['UE_flow_2'] if not pd.isna(row['UE_flow_2']) else 0
    
    if group in groups:  # 確保 group 存在於排序後的字典中
        groups[group]['UE_flow_1'] += ue_flow_1
        groups[group]['UE_flow_2'] += ue_flow_2

# 準備用於 MAPE 計算的列表
ue_se_values = []
actual_se_values = []
cube_se_values = []
ue_nw_values = []
actual_nw_values = []
cube_nw_values = []

results_comparison = []

print(f'——————————————————————詳細比較結果——————————————————————')

# 按照排序後的 group 順序處理
# UE_flow_1 代表(往南/東)流量， UE_flow_2 代表(往北/西)流量
for group in sorted_group_names:
    flows = groups[group]
    try:
        # 檢查 real_flow 中是否有該 group 的資料
        if group not in real_flow.index:
            print(f"【{group}】 資料不完整，跳過計算")
            continue
            
        # 安全地取得實際流量數據
        name = real_flow.loc[group, 'name'] if 'name' in real_flow.columns else f"Group {group}"
        south_east = real_flow.loc[group, '南/東'] if '南/東' in real_flow.columns and not pd.isna(real_flow.loc[group, '南/東']) else 1
        north_west = real_flow.loc[group, '北/西'] if '北/西' in real_flow.columns and not pd.isna(real_flow.loc[group, '北/西']) else 1
        
        # 計算差異百分比，避免除以零
        model_diff_se = (flows['UE_flow_1'] - south_east) / south_east * 100 if south_east != 0 else 0
        model_diff_nw = (flows['UE_flow_2'] - north_west) / north_west * 100 if north_west != 0 else 0
        
        # CUBE 差異計算（如果有 CUBE 資料）
        cube_se = real_flow.loc[group, 'CUBE南/東'] if 'CUBE南/東' in real_flow.columns and not pd.isna(real_flow.loc[group, 'CUBE南/東']) else south_east
        cube_nw = real_flow.loc[group, 'CUBE北/西'] if 'CUBE北/西' in real_flow.columns and not pd.isna(real_flow.loc[group, 'CUBE北/西']) else north_west
        
        cube_diff_se = (cube_se - south_east) / south_east * 100 if south_east != 0 else 0
        cube_diff_nw = (cube_nw - north_west) / north_west * 100 if north_west != 0 else 0
        
        # 計算 MAPE
        ue_mape_se = calculate_mape(south_east, flows['UE_flow_1'])
        ue_mape_nw = calculate_mape(north_west, flows['UE_flow_2'])
        cube_mape_se = calculate_mape(south_east, cube_se)
        cube_mape_nw = calculate_mape(north_west, cube_nw)
        
        # 收集數據用於整體 MAPE 計算
        if south_east > 0:  # 只計算非零實際值
            ue_se_values.append(flows['UE_flow_1'])
            actual_se_values.append(south_east)
            cube_se_values.append(cube_se)
            
        if north_west > 0:  # 只計算非零實際值
            ue_nw_values.append(flows['UE_flow_2'])
            actual_nw_values.append(north_west)
            cube_nw_values.append(cube_nw)
        
        # 儲存比較結果
        results_comparison.append({
            'group': group,
            'name': name,
            'actual_se': south_east,
            'ue_se': flows['UE_flow_1'],
            'cube_se': cube_se,
            'actual_nw': north_west,
            'ue_nw': flows['UE_flow_2'],
            'cube_nw': cube_nw,
            'ue_mape_se': ue_mape_se,
            'cube_mape_se': cube_mape_se,
            'ue_mape_nw': ue_mape_nw,
            'cube_mape_nw': cube_mape_nw
        })
        
        print(f"""【{group}】 {name}
    往南/東流量:
        實際流量 = {south_east:,.0f}
        UE 模型 = {round(flows['UE_flow_1']):,} (差異: {model_diff_se:+.2f}%, MAPE: {ue_mape_se:.2f}%)
        CUBE 模型 = {cube_se:,.0f} (差異: {cube_diff_se:+.2f}%, MAPE: {cube_mape_se:.2f}%)
    往北/西流量:
        實際流量 = {north_west:,.0f}
        UE 模型 = {round(flows['UE_flow_2']):,} (差異: {model_diff_nw:+.2f}%, MAPE: {ue_mape_nw:.2f}%)
        CUBE 模型 = {cube_nw:,.0f} (差異: {cube_diff_nw:+.2f}%, MAPE: {cube_mape_nw:.2f}%)
    """)
        
    except Exception as e:
        print(f"【{group}】 處理時發生錯誤: {e}")
        continue

# 計算整體 MAPE
print(f'\n{"="*80}')
print(f'整體模型表現評估 ({year}年{rush_hour})')
print(f'{"="*80}')

# 往南/東方向整體 MAPE
ue_overall_mape_se = calculate_overall_mape(actual_se_values, ue_se_values)
cube_overall_mape_se = calculate_overall_mape(actual_se_values, cube_se_values)

# 往北/西方向整體 MAPE
ue_overall_mape_nw = calculate_overall_mape(actual_nw_values, ue_nw_values)
cube_overall_mape_nw = calculate_overall_mape(actual_nw_values, cube_nw_values)

# 全方向整體 MAPE
all_actual = actual_se_values + actual_nw_values
all_ue = ue_se_values + ue_nw_values
all_cube = cube_se_values + cube_nw_values

ue_overall_mape = calculate_overall_mape(all_actual, all_ue)
cube_overall_mape = calculate_overall_mape(all_actual, all_cube)

print(f"""
往南/東方向 MAPE:
    UE 模型: {ue_overall_mape_se:.2f}%
    CUBE 模型: {cube_overall_mape_se:.2f}%
    差異: {ue_overall_mape_se - cube_overall_mape_se:+.2f}% (負值表示 UE 表現較好)

往北/西方向 MAPE:
    UE 模型: {ue_overall_mape_nw:.2f}%
    CUBE 模型: {cube_overall_mape_nw:.2f}%
    差異: {ue_overall_mape_nw - cube_overall_mape_nw:+.2f}% (負值表示 UE 表現較好)

全方向整體 MAPE:
    UE 模型: {ue_overall_mape:.2f}%
    CUBE 模型: {cube_overall_mape:.2f}%
    差異: {ue_overall_mape - cube_overall_mape:+.2f}% (負值表示 UE 表現較好)

統計摘要:
    評估路段數量: {len(results_comparison)}
    有效南/東流量比較: {len(actual_se_values)}
    有效北/西流量比較: {len(actual_nw_values)}
""")

# 創建詳細比較表格（按照 group 順序）
comparison_df = pd.DataFrame(results_comparison)
# 設定 group 欄位的排序順序
comparison_df['group'] = pd.Categorical(comparison_df['group'], categories=sorted_group_names, ordered=True)
comparison_df = comparison_df.sort_values('group')

print(f'\n{"="*80}')
print('詳細比較表格 (按 Group 順序)')
print(f'{"="*80}')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
print(comparison_df.round(2))

# 找出表現最好和最差的路段
if len(comparison_df) > 0:
    print(f'\n{"="*80}')
    print('模型表現分析')
    print(f'{"="*80}')
    
    # UE 模型表現最好的路段
    best_ue_se = comparison_df.loc[comparison_df['ue_mape_se'].idxmin()]
    worst_ue_se = comparison_df.loc[comparison_df['ue_mape_se'].idxmax()]
    
    print(f"""
UE 模型往南/東方向表現:
    最佳路段: 【{best_ue_se['group']}】{best_ue_se['name']} (MAPE: {best_ue_se['ue_mape_se']:.2f}%)
    最差路段: 【{worst_ue_se['group']}】{worst_ue_se['name']} (MAPE: {worst_ue_se['ue_mape_se']:.2f}%)
    
CUBE vs UE 比較:
    UE 表現較好的路段數量: {sum(comparison_df['ue_mape_se'] < comparison_df['cube_mape_se'])}
    CUBE 表現較好的路段數量: {sum(comparison_df['cube_mape_se'] < comparison_df['ue_mape_se'])}
    表現相同的路段數量: {sum(comparison_df['ue_mape_se'] == comparison_df['cube_mape_se'])}
    """)

# 顯示排序後的 sheet1 資料
print(f'\n{"="*80}')
print('Sheet1 資料 (按 Group 和 Link 排序)')
print(f'{"="*80}')

# 為 sheet1 添加排序功能
sheet1_copy = sheet1.copy()
sheet1_copy['group'] = pd.Categorical(sheet1_copy['group'], categories=sorted_group_names, ordered=True)
sheet1_sorted = sheet1_copy.sort_values(by=['group', 'link'])

pd.set_option('display.max_rows', None)
print(sheet1_sorted[["group", "link", "UE_flow_1", "UE_flow_2"]])

——————————————————————詳細比較結果——————————————————————
【SL1】 基隆河
    往南/東流量:
        實際流量 = 37,315
        UE 模型 = 35,026 (差異: -6.14%, MAPE: 6.14%)
        CUBE 模型 = 35,669 (差異: -4.41%, MAPE: 4.41%)
    往北/西流量:
        實際流量 = 32,441
        UE 模型 = 33,827 (差異: +4.27%, MAPE: 4.27%)
        CUBE 模型 = 31,701 (差異: -2.28%, MAPE: 2.28%)
    
【SL2】 台鐵(市民大道)
    往南/東流量:
        實際流量 = 33,288
        UE 模型 = 35,117 (差異: +5.50%, MAPE: 5.50%)
        CUBE 模型 = 35,864 (差異: +7.74%, MAPE: 7.74%)
    往北/西流量:
        實際流量 = 31,790
        UE 模型 = 31,594 (差異: -0.62%, MAPE: 0.62%)
        CUBE 模型 = 33,068 (差異: +4.02%, MAPE: 4.02%)
    
【SL3】 國1-市中心
    往南/東流量:
        實際流量 = 5,297
        UE 模型 = 5,272 (差異: -0.49%, MAPE: 0.49%)
        CUBE 模型 = 5,300 (差異: +0.06%, MAPE: 0.06%)
    往北/西流量:
        實際流量 = 5,489
        UE 模型 = 5,845 (差異: +6.48%, MAPE: 6.48%)
        CUBE 模型 = 4,411 (差異: -19.63%, MAPE: 19.63%)
    
【SL4】 建國南北路
    往南/東流量:
        實際流量 = 30,550
        UE 模型 = 23,859 (差異: -21.90%, MAPE: 21.90%)

In [2]:
# sheet1 sorted by group and link
sheet1.sort_values(by=['group', 'link'], inplace=True)
pd.set_option('display.max_rows', None)
sheet1[["group", "link", "UE_flow_1", "UE_flow_2"]]
#show all the results in a table

,group,link,UE_flow_1,UE_flow_2
4,SL1,中山橋,1177.489066,890.821673
13,SL1,南湖大橋,2058.973308,2029.584199
5,SL1,大直橋,2725.044232,2267.518978
12,SL1,成功橋,1835.079436,1532.810115
11,SL1,成美橋,2110.570914,1506.876892
2,SL1,承德橋,1239.744360,1796.148626
3,SL1,承德橋(機車專用道),960.835519,1252.950968
7,SL1,民權大橋,1621.971925,1475.566453
8,SL1,民權大橋(機車專用道),1276.815519,890.098052
0,SL1,洲美快速道路,3830.056303,4879.961973
